<a href="https://colab.research.google.com/github/NicKylis/letter_recognition/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import kagglehub
import shutil
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow import keras
from tensorflow.keras.utils import to_categorical

print ("hello world!")

hello world!


In [2]:
def get_MNIST_dataset(train_generator, batch_size=32, val_split=0.2, random_state=42):
  mnist = tf.keras.datasets.mnist
  (x_train, y_train), (x_test, y_test) = mnist.load_data()

  x_train = x_train.reshape(-1, 28, 28, 1).astype("float32") / 255.0
  x_test = x_test.reshape(-1, 28, 28, 1).astype("float32") / 255.0

  x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size = val_split, random_state=random_state)

  num_classes = 10  # MNIST has 10 classes (digits 0-9)
  y_train = to_categorical(y_train, num_classes)
  y_val = to_categorical(y_val, num_classes)
  y_test = to_categorical(y_test, num_classes)

  val_test_generator = ImageDataGenerator()
  train_gen = train_generator.flow(x_train, y_train, batch_size=batch_size)
  val_gen = val_test_generator.flow(x_val, y_val, batch_size=batch_size)
  test_gen = val_test_generator.flow(x_test, y_test, batch_size=batch_size, shuffle=False)

  return train_gen, val_gen, test_gen

In [3]:
train_generator = ImageDataGenerator()
train_generator_aug = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,)

train_data, val_data, test_data = get_MNIST_dataset(train_generator, batch_size=32, val_split=0.2, random_state=42)
train_data_aug, val_data_aug, test_data_aug = get_MNIST_dataset(train_generator, batch_size=32, val_split=0.2, random_state=42)

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


In [4]:
model = keras.Sequential([
    keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(28,28,1)),
    keras.layers.MaxPooling2D((2,2)),

    keras.layers.Conv2D(64, (3,3), activation='relu'),
    keras.layers.MaxPooling2D((2,2)),

    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(10, activation='softmax')  # 10 output neurons for digits 0-9
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [5]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)
history = model.fit(train_data_aug,
         validation_data=val_data_aug,
         epochs=20,
         batch_size=32,
         shuffle=True
         )#,callbacks=[callback])

Epoch 1/20


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1500/1500 ━━━━━━━━━━━━━━━━━━━━ 11s 4ms/step - accuracy: 0.9015 - loss: 0.3244 - val_accuracy: 0.9832 - val_loss: 0.0516
Epoch 2/20
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 9s 6ms/step - accuracy: 0.9864 - loss: 0.0437 - val_accuracy: 0.9884 - val_loss: 0.0396
Epoch 3/20
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 11s 7ms/step - accuracy: 0.9914 - loss: 0.0288 - val_accuracy: 0.9891 - val_loss: 0.0353
Epoch 4/20
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 17s 5ms/step - accuracy: 0.9936 - loss: 0.0207 - val_accuracy: 0.9872 - val_loss: 0.0456
Epoch 5/20
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.9951 - loss: 0.0148 - val_accuracy: 0.9858 - val_loss: 0.0540
Epoch 6/20
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.9967 - loss: 0.0105 - val_accuracy: 0.9893 - val_loss: 0.0489
Epoch 7/20
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 7s 5ms/step - accuracy: 0.9972 - loss: 0.0090 - val_accuracy: 0.9899 - val_loss: 0.0415
Epoch 8/20
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.9973 - loss: 0.0079 - val_accu

In [6]:
acc, loss = model.evaluate(test_data_aug)
print(f"Test Accuracy: {acc:.4f}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.9862 - loss: 0.0778


In [8]:
# for i in range(5):
#     plt.imshow(x_test[i].reshape(28, 28), cmap='gray')  # Reshape to 28x28 and plot
#     predicted_label = np.argmax(model.predict(x_test[i].reshape(1, 28, 28, 1)))
#     plt.title(f"True Label: {np.argmax(y_test[i])}, Predicted: {predicted_label}")
#     plt.show()

model.save("letter_recognition_model.keras")